<a href="https://colab.research.google.com/github/jacob-desselles/DS-Unit-1-Sprint-2-Statistics/blob/master/DS-Unit-1-Sprint-2-Statistics/Jacob_Desselles_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [13]:
### YOUR CODE STARTS HERE
from google.colab import files
df= files.upload()

Saving house-votes-84.data to house-votes-84 (1).data


In [15]:
import pandas as pd
df1 = pd.read_csv('house-votes-84.data')
df1

,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
430,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
431,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
432,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y


In [21]:
df1.isnull().sum()

republican    0
n             0
y             0
n.1           0
y.1           0
y.2           0
y.3           0
n.2           0
n.3           0
n.4           0
y.4           0
?             0
y.5           0
y.6           0
y.7           0
n.5           0
y.8           0
dtype: int64

In [26]:
column_headers = ['class', 'handicapped_infants','water_project_cost_sharing','adoption_of_the_budget_resolution',
                  'physician_fee_freeze','el_salvador_aid','religious_groups_in_schools','anti_satellite_test_ban',
                  'aid_to_nicaraguan_contras','mx_missile','immigration','synfuels_corporations_cutback','education_spending',
                  'superfund_right_to_sue','crime','duty_free_exports','export_administration_act_south_africa']

df1=pd.read_csv('house-votes-84.data',names=column_headers)
df1

,class,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporations_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y


In [28]:
df1.shape

(435, 17)

In [29]:
df1 = df1.replace('?',np.NaN)
df1

,class,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporations_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,NaN,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,NaN,NaN,NaN,NaN,n,y,y,y,n,y


For the purposes of making things easier for myself, I'll replace n's with 0 and y's with 1

In [30]:
df1= df1.replace('y',1)
df1

,class,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporations_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,n,1,n,1,1,1,n,n,n,1,NaN,1,1,1,n,1
1,republican,n,1,n,1,1,1,n,n,n,n,n,1,1,1,n,NaN
2,democrat,NaN,1,1,NaN,1,1,n,n,n,n,1,n,1,1,n,n
3,democrat,n,1,1,n,NaN,1,n,n,n,n,1,n,1,n,n,1
4,democrat,1,1,1,n,1,1,n,n,n,n,1,NaN,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,1,1,1,1,n,n,1,1,n,1,1,1,n,1
431,democrat,n,n,1,n,n,n,1,1,1,1,n,n,n,n,n,1
432,republican,n,NaN,n,1,1,1,n,n,n,n,1,1,1,1,n,1
433,republican,n,n,n,1,1,1,NaN,NaN,NaN,NaN,n,1,1,1,n,1


In [31]:
df1= df1.replace('n', 0)
df1

,class,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporations_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
431,democrat,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
432,republican,0.0,NaN,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
433,republican,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0


In [33]:
df1['class'].value_counts()

democrat      267
republican    168
Name: class, dtype: int64

In [34]:
libs=df1[df1['class']=='democrat']
caps=df1[df1['class']=='republican']
libs.head()

,class,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporations_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [35]:
caps.head()

,class,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporations_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
from scipy import stats

Null hypothesis: I believe that more republicans than democrats are in support of the superfund right to sue bill by at least 0.01 p.
Confidence level: 90%

In [38]:
stats.ttest_ind(libs['superfund_right_to_sue'],caps['superfund_right_to_sue'],nan_policy='omit')

Ttest_indResult(statistic=-13.51064251060933, pvalue=1.2278581709672758e-34)

In [39]:
caps['superfund_right_to_sue'].mean()

0.8607594936708861

In [40]:
libs['superfund_right_to_sue'].mean()

0.2896825396825397

In [41]:
stats.ttest_1samp(caps['superfund_right_to_sue'],.86,nan_policy='omit')

Ttest_1sampResult(statistic=0.027488451634497868, pvalue=0.9781050603046453)

In [42]:
stats.ttest_1samp(libs['superfund_right_to_sue'],.03,nan_policy='omit')

Ttest_1sampResult(statistic=9.069685042814198, pvalue=3.550375936245084e-17)

I have failed to reject my null hypothesis and can confirm more republicans than democrats support this issue. Moreso, it has broken the confidence level of at least 90% in the other direction.

Null hypothesis: I believe that more democrats will be in support of water project cost sharing than republicans will be by at least 10%.

Confidence level: 90%

In [50]:
stats.ttest_ind(libs['water_project_cost_sharing'],caps['water_project_cost_sharing'],nan_policy='omit')

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

In [51]:
caps['water_project_cost_sharing'].mean()

0.5067567567567568

In [52]:
libs['water_project_cost_sharing'].mean()

0.502092050209205

In [53]:
stats.ttest_1samp(caps['water_project_cost_sharing'],.51,nan_policy='omit')

Ttest_1sampResult(statistic=-0.07865165091579894, pvalue=0.9374166657416819)

In [54]:
stats.ttest_1samp(libs['water_project_cost_sharing'],.50,nan_policy='omit')

Ttest_1sampResult(statistic=0.06454972243678961, pvalue=0.9485867005339235)

Conclusion: I reject the null hypothesis, and, even more, say that I've answered the fourth question.

Null hypothesis: More democrats support the adoption of the budget resolution than republicans.

Confidence level: 90%

In [55]:
stats.ttest_ind(libs['adoption_of_the_budget_resolution'],caps['adoption_of_the_budget_resolution'],nan_policy='omit')

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

In [56]:
caps['adoption_of_the_budget_resolution'].mean()

0.13414634146341464

In [57]:
libs['adoption_of_the_budget_resolution'].mean()

0.8884615384615384

In [58]:
stats.ttest_1samp(caps['adoption_of_the_budget_resolution'],.13,nan_policy='omit')

Ttest_1sampResult(statistic=0.15532708869168346, pvalue=0.8767556431786082)

In [59]:
stats.ttest_1samp(libs['adoption_of_the_budget_resolution'],.89,nan_policy='omit')

Ttest_1sampResult(statistic=-0.07865111512328564, pvalue=0.9373708601919184)

I have failed to reject my null hypothesis and can confirm that more democrafts than republicans support the adoption of the budget resolution by at least 10 percent.

## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!